In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from retriever import SparseRetriever

sparse_retriever = SparseRetriever('enwiki-20171001-paragraph-5', ['10.60.0.59:9200'], timeout=60)

In [3]:
import redis

redis_client = redis.Redis(host='10.60.1.79', port=6379, db=3, password='redis4zyc', decode_responses=True)

In [7]:
from tqdm.auto import tqdm
from utils.data_utils import load_samples

RET_SIZE = 1000
hotpot_filter = {"term": {"for_hotpot": True}}

def batch_search(batch_q):
    batch_hits = [[hit['_id'] for hit in hits]
                  for hits in sparse_retriever.msearch(batch_q, RET_SIZE, filter_dic=hotpot_filter, n_retrieval=RET_SIZE * 2)]
    for query, hits in zip(batch_q, batch_hits):
        redis_client.delete(query)
        assert isinstance(hits, list), hits
        if len(hits) < RET_SIZE:
            redis_client.rpush(query, *(hits + ['EOL']))
        else:
            redis_client.rpush(query, *hits)

batch_size = 24
batch = []
for split in ['train', 'dev', 'test']:
    data_file = f'data/hotpot-{split}.tsv'
    for q_idx, (q_id, qas) in enumerate(tqdm(load_samples(data_file))):
        question = qas[0]
        if not redis_client.exists(question) or (redis_client.llen(question) < RET_SIZE and redis_client.lindex(question, -1) != 'EOL'):
            batch.append(question)
        if len(batch) == batch_size:
            batch_search(batch)
            batch = []
if len(batch) > 0:
    batch_search(batch)
    batch = []